In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.models import model_from_json

from skimage.transform import resize
from tensorflow.keras.optimizers import Adam


In [2]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
all_data = []

for i in test_dicoms: 
    dcm = pydicom.dcmread(i)
    fields = [dcm.PatientID, int(dcm.PatientAge), dcm.PatientSex, dcm.Modality, dcm.StudyDescription,
             dcm.Rows, dcm.Columns, dcm.BodyPartExamined, dcm.PatientPosition]
    all_data.append(fields)
mydata = pd.DataFrame(all_data, 
                      columns = ['PatientID','PatientAge','PatientSex','Modality','Findings','Rows','Columns','BodyPartExamined','PatientPosition'])

In [3]:
mydata

,PatientID,PatientAge,PatientSex,Modality,Findings,Rows,Columns,BodyPartExamined,PatientPosition
0,2,81,M,DX,No Finding,1024,1024,CHEST,PA
1,1,58,M,DX,Cardiomegaly,1024,1024,CHEST,AP
2,61,77,M,DX,Effusion,1024,1024,CHEST,AP
3,2,81,M,DX,No Finding,1024,1024,RIBCAGE,PA
4,2,81,M,CT,No Finding,1024,1024,CHEST,PA
5,2,81,M,DX,No Finding,1024,1024,CHEST,XX


In [4]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):    
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    # must be x-ray, and in age range
    if (ds.Modality == 'DX' and  int(ds.PatientAge) <= 95 and int(ds.PatientAge) >= 1 and 
        ds.BodyPartExamined == 'CHEST' and (ds.PatientPosition == 'PA' or ds.PatientPosition == 'AP')):
        return img
    else:
        print('This study is not compatible with the algorithm')
        return None
       
        
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # reduce size 
    img_res = resize(img, img_size)
    # add standardization 
    proc_img = (img_res - img_mean) / img_std   
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    with open(model_path, "r") as fp:
        model_json = fp.read()
    model = model_from_json(model_json)
    model.load_weights(weight_path)
    
    
    optimizer = Adam(lr=1e-4)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 

    pred = model.predict(img)
    
    if pred[0][0] > thresh:
        return  "Pneumonia"
    else:
        return  "No Pneumonia"
    return prediction 

In [6]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'pneunomia_weights.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

my_model =  load_model(model_path, weight_path) #loads model
thresh = 0.353 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_mean = np.mean(img) 
    img_std = np.std(img)
    
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Pneumonia
Load file test2.dcm ...
Pneumonia
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
This study is not compatible with the algorithm
Load file test5.dcm ...
This study is not compatible with the algorithm
Load file test6.dcm ...
This study is not compatible with the algorithm
